In [ ]:
!pip install pyautogen[gemini]
!pip install google-generativeai
!pip install ipywidgets
!pip install -q pyautogen google-generativeai python-dotenv ipywidgets
!pip install -q --upgrade pyautogen google-generativeai

In [ ]:
import os
from google.colab import userdata
import google.generativeai as genai
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time

# ===== CONFIGURATION =====
def setup_gemini():
    """Configure Gemini API using Colab secrets"""
    try:
        api_key = userdata.get('GEMINI_API_KEY')  # Get from Colab secrets
        genai.configure(api_key=api_key)
        return api_key
    except Exception as e:
        print("❌ Error: Please add GEMINI_API_KEY to Colab secrets")
        print("Go to 🔑 icon in left sidebar → Add new secret → Name: GEMINI_API_KEY")
        return None

def get_gemini_config(api_key: str, model: str = "gemini-1.5-flash"):
    """Get configuration for AutoGen agents"""
    return [{
        "model": model,
        "api_key": api_key,
        "api_type": "google",
        "base_url": "https://generativelanguage.googleapis.com/v1beta"
    }]

# ===== BMI CALCULATION TOOL =====
def calculate_bmi(weight_kg: float, height_cm: float) -> float:
    """Calculate BMI from weight and height"""
    height_m = height_cm / 100
    bmi = weight_kg / (height_m ** 2)
    return round(bmi, 1)

# ===== UI COMPONENTS =====
def create_ui():
    """Create simple UI for user input"""

    # Header
    display(HTML("""
    <div style='text-align: center; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                border-radius: 15px; margin-bottom: 20px; color: white;'>
        <h1 style='margin: 0; font-size: 2.2em;'>🤖 Smart Health Assistant</h1>
        <p style='margin: 10px 0 0 0; opacity: 0.9;'>Your personalized AI-powered health, diet, and fitness planner</p>
    </div>
    """))

    # Input widgets
    weight_widget = widgets.FloatSlider(
        value=70.0, min=30.0, max=200.0, step=0.1,
        description='Weight (kg):', style={'description_width': '120px'}
    )

    height_widget = widgets.IntSlider(
        value=170, min=100, max=250, step=1,
        description='Height (cm):', style={'description_width': '120px'}
    )

    age_widget = widgets.IntSlider(
        value=30, min=18, max=100, step=1,
        description='Age:', style={'description_width': '120px'}
    )

    gender_widget = widgets.Dropdown(
        options=['Male', 'Female', 'Other'],
        value='Male', description='Gender:', style={'description_width': '120px'}
    )

    diet_widget = widgets.Dropdown(
        options=['Veg', 'Non-Veg', 'Vegan'],
        value='Veg', description='Diet Preference:', style={'description_width': '120px'}
    )

    submit_button = widgets.Button(
        description='🚀 Generate Health Plan',
        button_style='success',
        layout=widgets.Layout(width='300px', height='45px')
    )

    # Layout
    input_box = widgets.VBox([
        widgets.HTML("<h3 style='color: #333;'>📝 Enter Your Health Details</h3>"),
        weight_widget, height_widget, age_widget, gender_widget, diet_widget,
        widgets.HTML("<br>"),
        submit_button
    ], layout=widgets.Layout(
        border='2px solid #e0e0e0', border_radius='10px',
        padding='20px', width='100%', max_width='500px'
    ))

    return input_box, submit_button, weight_widget, height_widget, age_widget, gender_widget, diet_widget

# ===== AGENT INITIALIZATION =====
def init_agents(api_key, user_data):
    """Initialize all AutoGen agents"""
    config_list = get_gemini_config(api_key)

    # BMI Agent
    bmi_agent = AssistantAgent(
        name="BMI_Agent",
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message=f"""You are a BMI specialist. Analyze BMI results and:
        1. Calculate BMI from weight ({user_data['weight']} kg) and height ({user_data['height']} cm)
        2. Categorize BMI (underweight <18.5, normal 18.5-24.9, overweight 25-29.9, obese ≥30)
        3. Provide health recommendations based on BMI category
        4. Be specific about health risks and benefits
        Always include the exact BMI value in your response."""
    )

    # Diet Planner Agent
    diet_agent = AssistantAgent(
        name="Diet_Planner",
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message=f"""You are a professional nutritionist. Create detailed meal plans based on:
        1. BMI analysis and recommendations from BMI_Agent
        2. Dietary preference: {user_data['diet']}
        3. User profile: {user_data['age']} years old, {user_data['gender']}

        Provide:
        - Breakfast, lunch, dinner, and 2 snacks
        - Portion sizes and calorie estimates
        - Specific food items based on dietary preference
        - Daily calorie target based on BMI recommendations"""
    )

    # Workout Scheduler Agent
    workout_agent = AssistantAgent(
        name="Workout_Scheduler",
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message=f"""You are a certified fitness trainer. Create weekly workout plans based on:
        1. Age: {user_data['age']} and gender: {user_data['gender']}
        2. BMI recommendations from BMI_Agent
        3. Meal plan suggestions from Diet_Planner

        Provide:
        - 7-day workout schedule
        - Mix of cardio, strength training, and flexibility
        - Duration and intensity for each workout
        - Rest days and recovery recommendations
        - Modifications based on BMI category"""
    )

    # User Proxy Agent
    user_proxy = UserProxyAgent(
        name="User_Proxy",
        human_input_mode="NEVER",
        code_execution_config=False,
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message="You collect and share user health data with other agents for personalized recommendations."
    )

    # Register BMI calculation function
    user_proxy.register_function(function_map={"calculate_bmi": calculate_bmi})

    return user_proxy, bmi_agent, diet_agent, workout_agent, config_list

# ===== MAIN PROCESSING =====
def process_health_plan(user_data, api_key):
    """Generate complete health plan using multi-agent system"""

    print("🤖 Initializing AI agents...")
    user_proxy, bmi_agent, diet_agent, workout_agent, config_list = init_agents(api_key, user_data)

    # Create group chat with sequential conversation
    groupchat = GroupChat(
        agents=[user_proxy, bmi_agent, diet_agent, workout_agent],
        messages=[],
        max_round=8,
        speaker_selection_method="round_robin"
    )

    manager = GroupChatManager(
        groupchat=groupchat,
        llm_config={"config_list": config_list, "cache_seed": None}
    )

    # Initial message with user data
    initial_message = f"""
    🏥 USER HEALTH PROFILE:
    ═══════════════════════════════════════
    📊 Basic Information:
    • Weight: {user_data['weight']} kg
    • Height: {user_data['height']} cm
    • Age: {user_data['age']} years
    • Gender: {user_data['gender']}

    🍽️ Dietary Preferences:
    • Preference: {user_data['diet']}

    ⚡ TASK SEQUENCE:
    1. BMI_Agent: Calculate BMI using calculate_bmi({user_data['weight']}, {user_data['height']}) and provide health analysis
    2. Diet_Planner: Create meal plan based on BMI analysis and dietary preference
    3. Workout_Scheduler: Design workout plan based on user profile and previous recommendations

    Please proceed with the health assessment in the specified sequence.
    """

    print("🔄 Generating your personalized health plan...")
    print("This may take 30-60 seconds...\n")

    # Start the conversation
    user_proxy.initiate_chat(
        manager,
        message=initial_message,
        clear_history=True
    )

    return groupchat.messages

# ===== DISPLAY RESULTS =====
def display_results(messages):
    """Display conversation results with nice formatting"""

    display(HTML("<hr style='border: 2px solid #667eea; margin: 20px 0;'>"))
    display(HTML("<h2 style='color: #667eea; text-align: center;'>🌟 Your Complete Health Plan</h2>"))

    # Process messages
    conversation = []
    final_plan = ""

    for msg in messages:
        if msg['role'] != 'system' and msg['content'].strip():
            agent_name = msg['name']
            content = msg['content']
            conversation.append((agent_name, content))

            # Store final workout plan
            if agent_name == "Workout_Scheduler":
                final_plan = content

    # Display each agent's response
    agent_icons = {
        "User_Proxy": "👤",
        "BMI_Agent": "📊",
        "Diet_Planner": "🍽️",
        "Workout_Scheduler": "💪"
    }

    for agent, message in conversation:
        icon = agent_icons.get(agent, "🤖")
        agent_display = agent.replace("_", " ")

        display(HTML(f"""
        <div style='border: 2px solid #e0e0e0; border-radius: 10px; margin: 15px 0; padding: 15px;
                    background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);'>
            <h3 style='color: #333; margin: 0 0 10px 0;'>{icon} {agent_display}</h3>
            <div style='background: white; padding: 15px; border-radius: 8px; white-space: pre-wrap;'>{message}</div>
        </div>
        """))

    # Save final plan
    if final_plan:
        # Create downloadable file
        full_plan = f"""
PERSONALIZED HEALTH PLAN
========================

{chr(10).join([f"{agent.replace('_', ' ').upper()}:{chr(10)}{'-'*50}{chr(10)}{message}{chr(10)}" for agent, message in conversation])}
        """

        # Save to file
        with open('personalized_health_plan.txt', 'w') as f:
            f.write(full_plan)

        display(HTML("""
        <div style='text-align: center; margin: 20px 0; padding: 15px;
                    background: #d4edda; border-radius: 10px; border: 2px solid #28a745;'>
            <h3 style='color: #155724; margin: 0;'>✅ Health Plan Generated Successfully!</h3>
            <p style='margin: 10px 0 0 0; color: #155724;'>Your plan has been saved as 'personalized_health_plan.txt'</p>
        </div>
        """))

        print("📁 Your health plan is saved in the files section (left sidebar)")

# ===== MAIN EXECUTION =====
def main():
    """Main function to run the Smart Health Assistant"""

    # Setup API
    api_key = setup_gemini()
    if not api_key:
        return

    # Create UI
    input_box, submit_button, weight_w, height_w, age_w, gender_w, diet_w = create_ui()
    display(input_box)

    # Handle button click
    def on_submit(b):
        clear_output()

        # Get user data
        user_data = {
            'weight': weight_w.value,
            'height': height_w.value,
            'age': age_w.value,
            'gender': gender_w.value,
            'diet': diet_w.value
        }

        # Display user data
        display(HTML(f"""
        <div style='background: #e8f4fd; border-radius: 10px; padding: 15px; margin: 10px 0; border: 2px solid #0084ff;'>
            <h3 style='color: #0056b3; margin: 0 0 10px 0;'>📋 Your Input Summary</h3>
            <p><strong>Weight:</strong> {user_data['weight']} kg | <strong>Height:</strong> {user_data['height']} cm |
               <strong>Age:</strong> {user_data['age']} | <strong>Gender:</strong> {user_data['gender']} |
               <strong>Diet:</strong> {user_data['diet']}</p>
        </div>
        """))

        try:
            # Process health plan
            messages = process_health_plan(user_data, api_key)

            # Display results
            display_results(messages)

        except Exception as e:
            display(HTML(f"""
            <div style='background: #f8d7da; border-radius: 10px; padding: 15px; margin: 10px 0; border: 2px solid #dc3545;'>
                <h3 style='color: #721c24; margin: 0 0 10px 0;'>❌ Error Occurred</h3>
                <p style='color: #721c24;'><strong>Error:</strong> {str(e)}</p>
                <p style='color: #721c24;'><strong>Please check:</strong> API key setup, internet connection, and try again.</p>
            </div>
            """))

    submit_button.on_click(on_submit)

# ===== RUN THE APPLICATION =====
if __name__ == "__main__":
    main()

In [ ]:
import os
from google.colab import userdata
import google.generativeai as genai
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time

# ===== CONFIGURATION =====
def setup_gemini():
    """Configure Gemini API using Colab secrets"""
    try:
        api_key = userdata.get('GEMINI_API_KEY')  # Get from Colab secrets
        genai.configure(api_key=api_key)
        return api_key
    except Exception as e:
        print("❌ Error: Please add GEMINI_API_KEY to Colab secrets")
        print("Go to 🔑 icon in left sidebar → Add new secret → Name: GEMINI_API_KEY")
        return None

def get_gemini_config(api_key: str, model: str = "gemini-1.5-flash"):
    """Get configuration for AutoGen agents"""
    return [{
        "model": model,
        "api_key": api_key,
        "api_type": "google",
        "base_url": "https://generativelanguage.googleapis.com/v1beta"
    }]

# ===== BMI CALCULATION TOOL =====
def calculate_bmi(weight_kg: float, height_cm: float) -> float:
    """Calculate BMI from weight and height"""
    height_m = height_cm / 100
    bmi = weight_kg / (height_m ** 2)
    return round(bmi, 1)

# ===== UI COMPONENTS =====
def create_ui():
    """Create simple UI for user input"""

    # Header
    display(HTML("""
    <div style='text-align: center; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                border-radius: 15px; margin-bottom: 20px; color: white;'>
        <h1 style='margin: 0; font-size: 2.2em;'>🤖 Smart Health Assistant</h1>
        <p style='margin: 10px 0 0 0; opacity: 0.9;'>Your personalized AI-powered health, diet, and fitness planner</p>
    </div>
    """))

    # Input widgets
    weight_widget = widgets.FloatSlider(
        value=70.0, min=30.0, max=200.0, step=0.1,
        description='Weight (kg):', style={'description_width': '120px'}
    )

    height_widget = widgets.IntSlider(
        value=170, min=100, max=250, step=1,
        description='Height (cm):', style={'description_width': '120px'}
    )

    age_widget = widgets.IntSlider(
        value=30, min=18, max=100, step=1,
        description='Age:', style={'description_width': '120px'}
    )

    gender_widget = widgets.Dropdown(
        options=['Male', 'Female', 'Other'],
        value='Male', description='Gender:', style={'description_width': '120px'}
    )

    diet_widget = widgets.Dropdown(
        options=['Veg', 'Non-Veg', 'Vegan'],
        value='Veg', description='Diet Preference:', style={'description_width': '120px'}
    )

    submit_button = widgets.Button(
        description='🚀 Generate Health Plan',
        button_style='success',
        layout=widgets.Layout(width='300px', height='45px')
    )

    # Layout
    input_box = widgets.VBox([
        widgets.HTML("<h3 style='color: #333;'>📝 Enter Your Health Details</h3>"),
        weight_widget, height_widget, age_widget, gender_widget, diet_widget,
        widgets.HTML("<br>"),
        submit_button
    ], layout=widgets.Layout(
        border='2px solid #e0e0e0', border_radius='10px',
        padding='20px', width='100%', max_width='500px'
    ))

    return input_box, submit_button, weight_widget, height_widget, age_widget, gender_widget, diet_widget

# ===== AGENT INITIALIZATION =====
def init_agents(api_key, user_data):
    """Initialize all AutoGen agents"""
    config_list = get_gemini_config(api_key)

    # BMI Agent
    bmi_agent = AssistantAgent(
        name="BMI_Agent",
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message=f"""You are a BMI specialist. Analyze BMI results and:
        1. Calculate BMI from weight ({user_data['weight']} kg) and height ({user_data['height']} cm)
        2. Categorize BMI (underweight <18.5, normal 18.5-24.9, overweight 25-29.9, obese ≥30)
        3. Provide health recommendations based on BMI category
        4. Be specific about health risks and benefits
        Always include the exact BMI value in your response."""
    )

    # Diet Planner Agent
    diet_agent = AssistantAgent(
        name="Diet_Planner",
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message=f"""You are a professional nutritionist. Create detailed meal plans based on:
        1. BMI analysis and recommendations from BMI_Agent
        2. Dietary preference: {user_data['diet']}
        3. User profile: {user_data['age']} years old, {user_data['gender']}

        Provide:
        - Breakfast, lunch, dinner, and 2 snacks
        - Portion sizes and calorie estimates
        - Specific food items based on dietary preference
        - Daily calorie target based on BMI recommendations"""
    )

    # Workout Scheduler Agent
    workout_agent = AssistantAgent(
        name="Workout_Scheduler",
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message=f"""You are a certified fitness trainer. Create weekly workout plans based on:
        1. Age: {user_data['age']} and gender: {user_data['gender']}
        2. BMI recommendations from BMI_Agent
        3. Meal plan suggestions from Diet_Planner

        Provide:
        - 7-day workout schedule
        - Mix of cardio, strength training, and flexibility
        - Duration and intensity for each workout
        - Rest days and recovery recommendations
        - Modifications based on BMI category"""
    )

    # User Proxy Agent
    user_proxy = UserProxyAgent(
        name="User_Proxy",
        human_input_mode="NEVER",
        code_execution_config=False,
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message="You collect and share user health data with other agents for personalized recommendations."
    )

    # Register BMI calculation function
    user_proxy.register_function(function_map={"calculate_bmi": calculate_bmi})

    return user_proxy, bmi_agent, diet_agent, workout_agent, config_list

# ===== MAIN PROCESSING =====
def process_health_plan(user_data, api_key):
    """Generate complete health plan using multi-agent system"""

    print("🤖 Initializing AI agents...")
    user_proxy, bmi_agent, diet_agent, workout_agent, config_list = init_agents(api_key, user_data)

    # Create group chat with sequential conversation
    groupchat = GroupChat(
        agents=[user_proxy, bmi_agent, diet_agent, workout_agent],
        messages=[],
        max_round=8,
        speaker_selection_method="round_robin"
    )

    manager = GroupChatManager(
        groupchat=groupchat,
        llm_config={"config_list": config_list, "cache_seed": None}
    )

    # Initial message with user data
    initial_message = f"""
    🏥 USER HEALTH PROFILE:
    ═══════════════════════════════════════
    📊 Basic Information:
    • Weight: {user_data['weight']} kg
    • Height: {user_data['height']} cm
    • Age: {user_data['age']} years
    • Gender: {user_data['gender']}

    🍽️ Dietary Preferences:
    • Preference: {user_data['diet']}

    ⚡ TASK SEQUENCE:
    1. BMI_Agent: Calculate BMI using calculate_bmi({user_data['weight']}, {user_data['height']}) and provide health analysis
    2. Diet_Planner: Create meal plan based on BMI analysis and dietary preference
    3. Workout_Scheduler: Design workout plan based on user profile and previous recommendations

    Please proceed with the health assessment in the specified sequence.
    """

    print("🔄 Generating your personalized health plan...")
    print("This may take 30-60 seconds...\n")

    # Start the conversation
    user_proxy.initiate_chat(
        manager,
        message=initial_message,
        clear_history=True
    )

    return groupchat.messages

# ===== DISPLAY RESULTS =====
def display_results(messages):
    """Display conversation results with nice formatting"""

    display(HTML("<hr style='border: 2px solid #667eea; margin: 20px 0;'>"))
    display(HTML("<h2 style='color: #667eea; text-align: center;'>🌟 Your Complete Health Plan</h2>"))

    # Process messages
    conversation = []
    final_plan = ""

    for msg in messages:
        if msg['role'] != 'system' and msg['content'].strip():
            agent_name = msg['name']
            content = msg['content']
            conversation.append((agent_name, content))

            # Store final workout plan
            if agent_name == "Workout_Scheduler":
                final_plan = content

    # Display each agent's response
    agent_icons = {
        "User_Proxy": "👤",
        "BMI_Agent": "📊",
        "Diet_Planner": "🍽️",
        "Workout_Scheduler": "💪"
    }

    for agent, message in conversation:
        icon = agent_icons.get(agent, "🤖")
        agent_display = agent.replace("_", " ")

        display(HTML(f"""
        <div style='border: 2px solid #e0e0e0; border-radius: 10px; margin: 15px 0; padding: 15px;
                    background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);'>
            <h3 style='color: #333; margin: 0 0 10px 0;'>{icon} {agent_display}</h3>
            <div style='background: white; padding: 15px; border-radius: 8px; white-space: pre-wrap;'>{message}</div>
        </div>
        """))

    # Save final plan
    if final_plan:
        # Create downloadable file
        full_plan = f"""
PERSONALIZED HEALTH PLAN
========================

{chr(10).join([f"{agent.replace('_', ' ').upper()}:{chr(10)}{'-'*50}{chr(10)}{message}{chr(10)}" for agent, message in conversation])}
        """

        # Save to file
        with open('personalized_health_plan.txt', 'w') as f:
            f.write(full_plan)

        display(HTML("""
        <div style='text-align: center; margin: 20px 0; padding: 15px;
                    background: #d4edda; border-radius: 10px; border: 2px solid #28a745;'>
            <h3 style='color: #155724; margin: 0;'>✅ Health Plan Generated Successfully!</h3>
            <p style='margin: 10px 0 0 0; color: #155724;'>Your plan has been saved as 'personalized_health_plan.txt'</p>
        </div>
        """))

        print("📁 Your health plan is saved in the files section (left sidebar)")

# ===== MAIN EXECUTION =====
def main():
    """Main function to run the Smart Health Assistant"""

    # Setup API
    api_key = setup_gemini()
    if not api_key:
        return

    # Create UI
    input_box, submit_button, weight_w, height_w, age_w, gender_w, diet_w = create_ui()
    display(input_box)

    # Handle button click
    def on_submit(b):
        clear_output()

        # Get user data
        user_data = {
            'weight': weight_w.value,
            'height': height_w.value,
            'age': age_w.value,
            'gender': gender_w.value,
            'diet': diet_w.value
        }

        # Display user data
        display(HTML(f"""
        <div style='background: #e8f4fd; border-radius: 10px; padding: 15px; margin: 10px 0; border: 2px solid #0084ff;'>
            <h3 style='color: #0056b3; margin: 0 0 10px 0;'>📋 Your Input Summary</h3>
            <p><strong>Weight:</strong> {user_data['weight']} kg | <strong>Height:</strong> {user_data['height']} cm |
               <strong>Age:</strong> {user_data['age']} | <strong>Gender:</strong> {user_data['gender']} |
               <strong>Diet:</strong> {user_data['diet']}</p>
        </div>
        """))

        try:
            # Process health plan
            messages = process_health_plan(user_data, api_key)

            # Display results
            display_results(messages)

        except Exception as e:
            display(HTML(f"""
            <div style='background: #f8d7da; border-radius: 10px; padding: 15px; margin: 10px 0; border: 2px solid #dc3545;'>
                <h3 style='color: #721c24; margin: 0 0 10px 0;'>❌ Error Occurred</h3>
                <p style='color: #721c24;'><strong>Error:</strong> {str(e)}</p>
                <p style='color: #721c24;'><strong>Please check:</strong> API key setup, internet connection, and try again.</p>
            </div>
            """))

    submit_button.on_click(on_submit)

# ===== RUN THE APPLICATION =====
if __name__ == "__main__":
    main()

In [ ]:
import os
from google.colab import userdata
import google.generativeai as genai
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time

# ===== CONFIGURATION =====
def setup_gemini():
    """Configure Gemini API using Colab secrets"""
    try:
        api_key = userdata.get('GEMINI_API_KEY')  # Get from Colab secrets
        genai.configure(api_key=api_key)
        return api_key
    except Exception as e:
        print("❌ Error: Please add GEMINI_API_KEY to Colab secrets")
        print("Go to 🔑 icon in left sidebar → Add new secret → Name: GEMINI_API_KEY")
        return None

def get_gemini_config(api_key: str, model: str = "gemini-1.5-flash"):
    """Get configuration for AutoGen agents"""
    return [{
        "model": model,
        "api_key": api_key,
        "api_type": "google",
        "base_url": "https://generativelanguage.googleapis.com/v1beta"
    }]

# ===== BMI CALCULATION TOOL =====
def calculate_bmi(weight_kg: float, height_cm: float) -> float:
    """Calculate BMI from weight and height"""
    height_m = height_cm / 100
    bmi = weight_kg / (height_m ** 2)
    return round(bmi, 1)

# ===== UI COMPONENTS =====
def create_ui():
    """Create simple UI for user input"""

    # Header
    display(HTML("""
    <div style='text-align: center; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                border-radius: 15px; margin-bottom: 20px; color: white;'>
        <h1 style='margin: 0; font-size: 2.2em;'>🤖 Smart Health Assistant</h1>
        <p style='margin: 10px 0 0 0; opacity: 0.9;'>Your personalized AI-powered health, diet, and fitness planner</p>
    </div>
    """))

    # Input widgets
    weight_widget = widgets.FloatSlider(
        value=70.0, min=30.0, max=200.0, step=0.1,
        description='Weight (kg):', style={'description_width': '120px'}
    )

    height_widget = widgets.IntSlider(
        value=170, min=100, max=250, step=1,
        description='Height (cm):', style={'description_width': '120px'}
    )

    age_widget = widgets.IntSlider(
        value=30, min=18, max=100, step=1,
        description='Age:', style={'description_width': '120px'}
    )

    gender_widget = widgets.Dropdown(
        options=['Male', 'Female', 'Other'],
        value='Male', description='Gender:', style={'description_width': '120px'}
    )

    diet_widget = widgets.Dropdown(
        options=['Veg', 'Non-Veg', 'Vegan'],
        value='Veg', description='Diet Preference:', style={'description_width': '120px'}
    )

    submit_button = widgets.Button(
        description='🚀 Generate Health Plan',
        button_style='success',
        layout=widgets.Layout(width='300px', height='45px')
    )

    # Layout
    input_box = widgets.VBox([
        widgets.HTML("<h3 style='color: #333;'>📝 Enter Your Health Details</h3>"),
        weight_widget, height_widget, age_widget, gender_widget, diet_widget,
        widgets.HTML("<br>"),
        submit_button
    ], layout=widgets.Layout(
        border='2px solid #e0e0e0', border_radius='10px',
        padding='20px', width='100%', max_width='500px'
    ))

    return input_box, submit_button, weight_widget, height_widget, age_widget, gender_widget, diet_widget

# ===== AGENT INITIALIZATION =====
def init_agents(api_key, user_data):
    """Initialize all AutoGen agents"""
    config_list = get_gemini_config(api_key)

    # BMI Agent
    bmi_agent = AssistantAgent(
        name="BMI_Agent",
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message=f"""You are a BMI specialist. Analyze BMI results and:
        1. Calculate BMI from weight ({user_data['weight']} kg) and height ({user_data['height']} cm)
        2. Categorize BMI (underweight <18.5, normal 18.5-24.9, overweight 25-29.9, obese ≥30)
        3. Provide health recommendations based on BMI category
        4. Be specific about health risks and benefits
        Always include the exact BMI value in your response."""
    )

    # Diet Planner Agent
    diet_agent = AssistantAgent(
        name="Diet_Planner",
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message=f"""You are a professional nutritionist. Create detailed meal plans based on:
        1. BMI analysis and recommendations from BMI_Agent
        2. Dietary preference: {user_data['diet']}
        3. User profile: {user_data['age']} years old, {user_data['gender']}

        Provide:
        - Breakfast, lunch, dinner, and 2 snacks
        - Portion sizes and calorie estimates
        - Specific food items based on dietary preference
        - Daily calorie target based on BMI recommendations"""
    )

    # Workout Scheduler Agent
    workout_agent = AssistantAgent(
        name="Workout_Scheduler",
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message=f"""You are a certified fitness trainer. Create weekly workout plans based on:
        1. Age: {user_data['age']} and gender: {user_data['gender']}
        2. BMI recommendations from BMI_Agent
        3. Meal plan suggestions from Diet_Planner

        Provide:
        - 7-day workout schedule
        - Mix of cardio, strength training, and flexibility
        - Duration and intensity for each workout
        - Rest days and recovery recommendations
        - Modifications based on BMI category"""
    )

    # User Proxy Agent
    user_proxy = UserProxyAgent(
        name="User_Proxy",
        human_input_mode="NEVER",
        code_execution_config=False,
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message="You collect and share user health data with other agents for personalized recommendations."
    )

    # Register BMI calculation function
    user_proxy.register_function(function_map={"calculate_bmi": calculate_bmi})

    return user_proxy, bmi_agent, diet_agent, workout_agent, config_list

# ===== MAIN PROCESSING =====
def process_health_plan(user_data, api_key):
    """Generate complete health plan using multi-agent system"""

    print("🤖 Initializing AI agents...")
    user_proxy, bmi_agent, diet_agent, workout_agent, config_list = init_agents(api_key, user_data)

    # Create group chat with sequential conversation
    groupchat = GroupChat(
        agents=[user_proxy, bmi_agent, diet_agent, workout_agent],
        messages=[],
        max_round=8,
        speaker_selection_method="round_robin"
    )

    manager = GroupChatManager(
        groupchat=groupchat,
        llm_config={"config_list": config_list, "cache_seed": None}
    )

    # Initial message with user data
    initial_message = f"""
    🏥 USER HEALTH PROFILE:
    ═══════════════════════════════════════
    📊 Basic Information:
    • Weight: {user_data['weight']} kg
    • Height: {user_data['height']} cm
    • Age: {user_data['age']} years
    • Gender: {user_data['gender']}

    🍽️ Dietary Preferences:
    • Preference: {user_data['diet']}

    ⚡ TASK SEQUENCE:
    1. BMI_Agent: Calculate BMI using calculate_bmi({user_data['weight']}, {user_data['height']}) and provide health analysis
    2. Diet_Planner: Create meal plan based on BMI analysis and dietary preference
    3. Workout_Scheduler: Design workout plan based on user profile and previous recommendations

    Please proceed with the health assessment in the specified sequence.
    """

    print("🔄 Generating your personalized health plan...")
    print("This may take 30-60 seconds...\n")

    # Start the conversation
    user_proxy.initiate_chat(
        manager,
        message=initial_message,
        clear_history=True
    )

    return groupchat.messages

# ===== DISPLAY RESULTS =====
def display_results(messages):
    """Display conversation results with nice formatting"""

    display(HTML("<hr style='border: 2px solid #667eea; margin: 20px 0;'>"))
    display(HTML("<h2 style='color: #667eea; text-align: center;'>🌟 Your Complete Health Plan</h2>"))

    # Process messages
    conversation = []
    final_plan = ""

    for msg in messages:
        if msg['role'] != 'system' and msg['content'].strip():
            agent_name = msg['name']
            content = msg['content']
            conversation.append((agent_name, content))

            # Store final workout plan
            if agent_name == "Workout_Scheduler":
                final_plan = content

    # Display each agent's response
    agent_icons = {
        "User_Proxy": "👤",
        "BMI_Agent": "📊",
        "Diet_Planner": "🍽️",
        "Workout_Scheduler": "💪"
    }

    for agent, message in conversation:
        icon = agent_icons.get(agent, "🤖")
        agent_display = agent.replace("_", " ")

        display(HTML(f"""
        <div style='border: 2px solid #e0e0e0; border-radius: 10px; margin: 15px 0; padding: 15px;
                    background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);'>
            <h3 style='color: #333; margin: 0 0 10px 0;'>{icon} {agent_display}</h3>
            <div style='background: white; padding: 15px; border-radius: 8px; white-space: pre-wrap;'>{message}</div>
        </div>
        """))

    # Save final plan
    if final_plan:
        # Create downloadable file
        full_plan = f"""
PERSONALIZED HEALTH PLAN
========================

{chr(10).join([f"{agent.replace('_', ' ').upper()}:{chr(10)}{'-'*50}{chr(10)}{message}{chr(10)}" for agent, message in conversation])}
        """

        # Save to file
        with open('personalized_health_plan.txt', 'w') as f:
            f.write(full_plan)

        display(HTML("""
        <div style='text-align: center; margin: 20px 0; padding: 15px;
                    background: #d4edda; border-radius: 10px; border: 2px solid #28a745;'>
            <h3 style='color: #155724; margin: 0;'>✅ Health Plan Generated Successfully!</h3>
            <p style='margin: 10px 0 0 0; color: #155724;'>Your plan has been saved as 'personalized_health_plan.txt'</p>
        </div>
        """))

        print("📁 Your health plan is saved in the files section (left sidebar)")

# ===== MAIN EXECUTION =====
def main():
    """Main function to run the Smart Health Assistant"""

    # Setup API
    api_key = setup_gemini()
    if not api_key:
        return

    # Create UI
    input_box, submit_button, weight_w, height_w, age_w, gender_w, diet_w = create_ui()
    display(input_box)

    # Handle button click
    def on_submit(b):
        clear_output()

        # Get user data
        user_data = {
            'weight': weight_w.value,
            'height': height_w.value,
            'age': age_w.value,
            'gender': gender_w.value,
            'diet': diet_w.value
        }

        # Display user data
        display(HTML(f"""
        <div style='background: #e8f4fd; border-radius: 10px; padding: 15px; margin: 10px 0; border: 2px solid #0084ff;'>
            <h3 style='color: #0056b3; margin: 0 0 10px 0;'>📋 Your Input Summary</h3>
            <p><strong>Weight:</strong> {user_data['weight']} kg | <strong>Height:</strong> {user_data['height']} cm |
               <strong>Age:</strong> {user_data['age']} | <strong>Gender:</strong> {user_data['gender']} |
               <strong>Diet:</strong> {user_data['diet']}</p>
        </div>
        """))

        try:
            # Process health plan
            messages = process_health_plan(user_data, api_key)

            # Display results
            display_results(messages)

        except Exception as e:
            display(HTML(f"""
            <div style='background: #f8d7da; border-radius: 10px; padding: 15px; margin: 10px 0; border: 2px solid #dc3545;'>
                <h3 style='color: #721c24; margin: 0 0 10px 0;'>❌ Error Occurred</h3>
                <p style='color: #721c24;'><strong>Error:</strong> {str(e)}</p>
                <p style='color: #721c24;'><strong>Please check:</strong> API key setup, internet connection, and try again.</p>
            </div>
            """))

    submit_button.on_click(on_submit)

# ===== RUN THE APPLICATION =====
if __name__ == "__main__":
    main()

In [ ]:
import os
from google.colab import userdata
import google.generativeai as genai
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time

# ===== CONFIGURATION =====
def setup_gemini():
    """Configure Gemini API using Colab secrets"""
    try:
        api_key = userdata.get('GEMINI_API_KEY')  # Get from Colab secrets
        genai.configure(api_key=api_key)
        return api_key
    except Exception as e:
        print("❌ Error: Please add GEMINI_API_KEY to Colab secrets")
        print("Go to 🔑 icon in left sidebar → Add new secret → Name: GEMINI_API_KEY")
        return None

def get_gemini_config(api_key: str, model: str = "gemini-1.5-flash"):
    """Get configuration for AutoGen agents"""
    return [{
        "model": model,
        "api_key": api_key,
        "api_type": "google",
        "base_url": "https://generativelanguage.googleapis.com/v1beta"
    }]

# ===== BMI CALCULATION TOOL =====
def calculate_bmi(weight_kg: float, height_cm: float) -> float:
    """Calculate BMI from weight and height"""
    height_m = height_cm / 100
    bmi = weight_kg / (height_m ** 2)
    return round(bmi, 1)

# ===== UI COMPONENTS =====
def create_ui():
    """Create simple UI for user input"""

    # Header
    display(HTML("""
    <div style='text-align: center; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                border-radius: 15px; margin-bottom: 20px; color: white;'>
        <h1 style='margin: 0; font-size: 2.2em;'>🤖 Smart Health Assistant</h1>
        <p style='margin: 10px 0 0 0; opacity: 0.9;'>Your personalized AI-powered health, diet, and fitness planner</p>
    </div>
    """))

    # Input widgets
    weight_widget = widgets.FloatSlider(
        value=70.0, min=30.0, max=200.0, step=0.1,
        description='Weight (kg):', style={'description_width': '120px'}
    )

    height_widget = widgets.IntSlider(
        value=170, min=100, max=250, step=1,
        description='Height (cm):', style={'description_width': '120px'}
    )

    age_widget = widgets.IntSlider(
        value=30, min=18, max=100, step=1,
        description='Age:', style={'description_width': '120px'}
    )

    gender_widget = widgets.Dropdown(
        options=['Male', 'Female', 'Other'],
        value='Male', description='Gender:', style={'description_width': '120px'}
    )

    diet_widget = widgets.Dropdown(
        options=['Veg', 'Non-Veg', 'Vegan'],
        value='Veg', description='Diet Preference:', style={'description_width': '120px'}
    )

    submit_button = widgets.Button(
        description='🚀 Generate Health Plan',
        button_style='success',
        layout=widgets.Layout(width='300px', height='45px')
    )

    # Layout
    input_box = widgets.VBox([
        widgets.HTML("<h3 style='color: #333;'>📝 Enter Your Health Details</h3>"),
        weight_widget, height_widget, age_widget, gender_widget, diet_widget,
        widgets.HTML("<br>"),
        submit_button
    ], layout=widgets.Layout(
        border='2px solid #e0e0e0', border_radius='10px',
        padding='20px', width='100%', max_width='500px'
    ))

    return input_box, submit_button, weight_widget, height_widget, age_widget, gender_widget, diet_widget

# ===== AGENT INITIALIZATION =====
def init_agents(api_key, user_data):
    """Initialize all AutoGen agents"""
    config_list = get_gemini_config(api_key)

    # BMI Agent
    bmi_agent = AssistantAgent(
        name="BMI_Agent",
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message=f"""You are a BMI specialist. Analyze BMI results and:
        1. Calculate BMI from weight ({user_data['weight']} kg) and height ({user_data['height']} cm)
        2. Categorize BMI (underweight <18.5, normal 18.5-24.9, overweight 25-29.9, obese ≥30)
        3. Provide health recommendations based on BMI category
        4. Be specific about health risks and benefits
        Always include the exact BMI value in your response."""
    )

    # Diet Planner Agent
    diet_agent = AssistantAgent(
        name="Diet_Planner",
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message=f"""You are a professional nutritionist. Create detailed meal plans based on:
        1. BMI analysis and recommendations from BMI_Agent
        2. Dietary preference: {user_data['diet']}
        3. User profile: {user_data['age']} years old, {user_data['gender']}

        Provide:
        - Breakfast, lunch, dinner, and 2 snacks
        - Portion sizes and calorie estimates
        - Specific food items based on dietary preference
        - Daily calorie target based on BMI recommendations"""
    )

    # Workout Scheduler Agent
    workout_agent = AssistantAgent(
        name="Workout_Scheduler",
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message=f"""You are a certified fitness trainer. Create weekly workout plans based on:
        1. Age: {user_data['age']} and gender: {user_data['gender']}
        2. BMI recommendations from BMI_Agent
        3. Meal plan suggestions from Diet_Planner

        Provide:
        - 7-day workout schedule
        - Mix of cardio, strength training, and flexibility
        - Duration and intensity for each workout
        - Rest days and recovery recommendations
        - Modifications based on BMI category"""
    )

    # User Proxy Agent
    user_proxy = UserProxyAgent(
        name="User_Proxy",
        human_input_mode="NEVER",
        code_execution_config=False,
        llm_config={"config_list": config_list, "cache_seed": None},
        system_message="You collect and share user health data with other agents for personalized recommendations."
    )

    # Register BMI calculation function
    user_proxy.register_function(function_map={"calculate_bmi": calculate_bmi})

    return user_proxy, bmi_agent, diet_agent, workout_agent, config_list

# ===== MAIN PROCESSING =====
def process_health_plan(user_data, api_key):
    """Generate complete health plan using multi-agent system"""

    print("🤖 Initializing AI agents...")
    user_proxy, bmi_agent, diet_agent, workout_agent, config_list = init_agents(api_key, user_data)

    # Create group chat with sequential conversation
    groupchat = GroupChat(
        agents=[user_proxy, bmi_agent, diet_agent, workout_agent],
        messages=[],
        max_round=8,
        speaker_selection_method="round_robin"
    )

    manager = GroupChatManager(
        groupchat=groupchat,
        llm_config={"config_list": config_list, "cache_seed": None}
    )

    # Initial message with user data
    initial_message = f"""
    🏥 USER HEALTH PROFILE:
    ═══════════════════════════════════════
    📊 Basic Information:
    • Weight: {user_data['weight']} kg
    • Height: {user_data['height']} cm
    • Age: {user_data['age']} years
    • Gender: {user_data['gender']}

    🍽️ Dietary Preferences:
    • Preference: {user_data['diet']}

    ⚡ TASK SEQUENCE:
    1. BMI_Agent: Calculate BMI using calculate_bmi({user_data['weight']}, {user_data['height']}) and provide health analysis
    2. Diet_Planner: Create meal plan based on BMI analysis and dietary preference
    3. Workout_Scheduler: Design workout plan based on user profile and previous recommendations

    Please proceed with the health assessment in the specified sequence.
    """

    print("🔄 Generating your personalized health plan...")
    print("This may take 30-60 seconds...\n")

    # Start the conversation
    user_proxy.initiate_chat(
        manager,
        message=initial_message,
        clear_history=True
    )

    return groupchat.messages

# ===== DISPLAY RESULTS =====
def display_results(messages):
    """Display conversation results with nice formatting"""

    display(HTML("<hr style='border: 2px solid #667eea; margin: 20px 0;'>"))
    display(HTML("<h2 style='color: #667eea; text-align: center;'>🌟 Your Complete Health Plan</h2>"))

    # Process messages
    conversation = []
    final_plan = ""

    for msg in messages:
        if msg['role'] != 'system' and msg['content'].strip():
            agent_name = msg['name']
            content = msg['content']
            conversation.append((agent_name, content))

            # Store final workout plan
            if agent_name == "Workout_Scheduler":
                final_plan = content

    # Display each agent's response
    agent_icons = {
        "User_Proxy": "👤",
        "BMI_Agent": "📊",
        "Diet_Planner": "🍽️",
        "Workout_Scheduler": "💪"
    }

    for agent, message in conversation:
        icon = agent_icons.get(agent, "🤖")
        agent_display = agent.replace("_", " ")

        display(HTML(f"""
        <div style='border: 2px solid #e0e0e0; border-radius: 10px; margin: 15px 0; padding: 15px;
                    background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);'>
            <h3 style='color: #333; margin: 0 0 10px 0;'>{icon} {agent_display}</h3>
            <div style='background: white; padding: 15px; border-radius: 8px; white-space: pre-wrap;'>{message}</div>
        </div>
        """))

    # Save final plan
    if final_plan:
        # Create downloadable file
        full_plan = f"""
PERSONALIZED HEALTH PLAN
========================

{chr(10).join([f"{agent.replace('_', ' ').upper()}:{chr(10)}{'-'*50}{chr(10)}{message}{chr(10)}" for agent, message in conversation])}
        """

        # Save to file
        with open('personalized_health_plan.txt', 'w') as f:
            f.write(full_plan)

        display(HTML("""
        <div style='text-align: center; margin: 20px 0; padding: 15px;
                    background: #d4edda; border-radius: 10px; border: 2px solid #28a745;'>
            <h3 style='color: #155724; margin: 0;'>✅ Health Plan Generated Successfully!</h3>
            <p style='margin: 10px 0 0 0; color: #155724;'>Your plan has been saved as 'personalized_health_plan.txt'</p>
        </div>
        """))

        print("📁 Your health plan is saved in the files section (left sidebar)")

# ===== MAIN EXECUTION =====
def main():
    """Main function to run the Smart Health Assistant"""

    # Setup API
    api_key = setup_gemini()
    if not api_key:
        return

    # Create UI
    input_box, submit_button, weight_w, height_w, age_w, gender_w, diet_w = create_ui()
    display(input_box)

    # Handle button click
    def on_submit(b):
        clear_output()

        # Get user data
        user_data = {
            'weight': weight_w.value,
            'height': height_w.value,
            'age': age_w.value,
            'gender': gender_w.value,
            'diet': diet_w.value
        }

        # Display user data
        display(HTML(f"""
        <div style='background: #e8f4fd; border-radius: 10px; padding: 15px; margin: 10px 0; border: 2px solid #0084ff;'>
            <h3 style='color: #0056b3; margin: 0 0 10px 0;'>📋 Your Input Summary</h3>
            <p><strong>Weight:</strong> {user_data['weight']} kg | <strong>Height:</strong> {user_data['height']} cm |
               <strong>Age:</strong> {user_data['age']} | <strong>Gender:</strong> {user_data['gender']} |
               <strong>Diet:</strong> {user_data['diet']}</p>
        </div>
        """))

        try:
            # Process health plan
            messages = process_health_plan(user_data, api_key)

            # Display results
            display_results(messages)

        except Exception as e:
            display(HTML(f"""
            <div style='background: #f8d7da; border-radius: 10px; padding: 15px; margin: 10px 0; border: 2px solid #dc3545;'>
                <h3 style='color: #721c24; margin: 0 0 10px 0;'>❌ Error Occurred</h3>
                <p style='color: #721c24;'><strong>Error:</strong> {str(e)}</p>
                <p style='color: #721c24;'><strong>Please check:</strong> API key setup, internet connection, and try again.</p>
            </div>
            """))

    submit_button.on_click(on_submit)

# ===== RUN THE APPLICATION =====
if __name__ == "__main__":
    main()